In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import time
import os
import glob
from IPython import display as ipydisplay
import threading
import queue
from flask import Flask, render_template, Response, request, jsonify
import json

mp_pose = mp.solutions.pose

RTSP_URL = "rtsp://127.0.0.1:8554/webcam"

# Suppress H.264 warnings
os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = (
    "rtsp_transport;tcp|"
    "fflags;nobuffer+discardcorrupt|"
    "flags;low_delay|"
    "max_delay;0|"
    "probesize;32|"
    "analyzeduration;0|"
    "err_detect;ignore_err"
)

## 1. Extract Reference (Run Once)

In [ ]:
def extract_all_references_from_folder(folder='exercises', target_fps=10.0):
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"Created {folder}/ – add your .mp4 files")
        return

    video_files = [f for f in glob.glob(os.path.join(folder, "*.*")) if f.lower().endswith(('.mp4', '.avi', '.mov'))]
    if not video_files:
        print("No videos in exercises/")
        return

    pose = mp_pose.Pose(static_image_mode=False, model_complexity=1)
    for video_path in video_files:
        name = os.path.splitext(os.path.basename(video_path))[0]
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS) or 30
        interval = max(1, int(fps / target_fps))
        refs, cnt = [], 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break
            if cnt % interval == 0:
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                res = pose.process(rgb)
                if res.pose_landmarks:
                    refs.append([(l.x, l.y, l.z, l.visibility) for l in res.pose_landmarks.landmark])
            cnt += 1
        cap.release()
        with open(f"{name}.pkl", 'wb') as f:
            pickle.dump(refs, f)
        print(f"{name}.pkl → {len(refs)} frames")
    pose.close()
    print("Done!")

extract_all_references_from_folder()

## 2. Flask Web Server + MJPEG Stream

In [ ]:
# --------------------------------------------------------------
#  Flask GUI – fixed version (no templates, safe thread, logging)
# --------------------------------------------------------------
import logging
from flask import Flask, Response, request, jsonify
import threading
import queue
import json
import os

# ------------------------------------------------------------------
app = Flask(__name__)
logging.basicConfig(level=logging.INFO)   # show errors in notebook

stream_active = False
current_exercise = None
frame_queue = queue.Queue(maxsize=2)

def get_available_exercises():
    return [os.path.splitext(f)[0] for f in os.listdir('.') if f.endswith('.pkl')]

# --------------------------------------------------  HTML page  -----
# --------------------------------------------------  HTML page  -----
HTML_PAGE = """<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Pose Form Checker</title>
    <style>
        body{font-family:Arial;text-align:center;margin:2rem;background:#fafafa;}
        h1{color:#333;}
        select,button{font-size:1.2rem;margin:0.5rem;padding:0.5rem;}
        #video{max-width:90%;border:3px solid #444;display:none;}
        .status{font-weight:bold;color:#555;margin-top:1rem;}
        .nav-buttons { margin: 1rem 0; }
        .nav-btn {
            background: #2563eb; color: white; border: none; padding: 0.7rem 1.2rem;
            border-radius: 8px; cursor: pointer; font-size: 1rem; margin: 0.3rem;
            text-decoration: none; display: inline-block;
        }
        .nav-btn:hover { background: #1d4ed8; }
    </style>
</head>
<body>
    <h1>Pose Form Checker</h1>
    
    <div class="nav-buttons">
        <a href="http://10.227.207.170:3001/" target="_blank" class="nav-btn">Go to Bulk Predictor</a>
    </div>

    <select id="exercise">
        {% for ex in exercises %}
        <option value="{{ ex }}">{{ ex.replace('_',' ') | title }}</option>
        {% endfor %}
    </select><br>
    <button onclick="start()">Start</button>
    <button onclick="stop()" disabled>Stop</button>
    <p class="status" id="status">Ready – pick an exercise</p>
    <img id="video" src="">

    <script>
    const img = document.getElementById('video');
    function start(){
        const ex = document.getElementById('exercise').value;
        fetch('/start',{
            method:'POST',
            headers:{'Content-Type':'application/json'},
            body:JSON.stringify({exercise:ex})
        })
        .then(r=>r.json())
        .then(d=>{
            if(d.status==='started'){
                img.src='/video_feed';
                img.style.display='block';
                document.querySelector('button[onclick="start()"]').disabled=true;
                document.querySelector('button[onclick="stop()"]').disabled=false;
                document.getElementById('status').textContent='Live: '+ex.replace(/_/g,' ');
            }
        });
    }
    function stop(){
        fetch('/stop',{method:'POST'}).then(()=>{
            img.src=''; img.style.display='none';
            document.querySelector('button[onclick="start()"]').disabled=false;
            document.querySelector('button[onclick="stop()"]').disabled=true;
            document.getElementById('status').textContent='Stopped';
        });
    }
    </script>
</body>
</html>"""
# --------------------------------------------------  Routes  -----
@app.route('/')
def index():
    return app.jinja_env.from_string(HTML_PAGE).render(exercises=get_available_exercises())

@app.route('/start', methods=['POST'])
def start():
    global stream_active, current_exercise
    try:
        data = request.get_json(silent=True) or {}
        ex = data.get('exercise')
        if ex and os.path.exists(f"{ex}.pkl"):
            current_exercise = ex
            stream_active = True
            threading.Thread(target=stream_worker, daemon=True).start()
            return jsonify(status='started')
        return jsonify(status='error', message='Exercise not found')
    except Exception as e:
        logging.exception("START error")
        return jsonify(status='error', message=str(e))

@app.route('/stop', methods=['POST'])
def stop():
    global stream_active
    stream_active = False
    return jsonify(status='stopped')

# --------------------------------------------------  Worker  -----
def stream_worker():
    global frame_queue, stream_active, current_exercise
    if not current_exercise: return
    try:
        with open(f"{current_exercise}.pkl", 'rb') as f:
            ref_seq = pickle.load(f)
    except Exception as e:
        logging.error(f"Failed to load {current_exercise}.pkl: {e}")
        stream_active = False
        return

    pose = mp_pose.Pose(static_image_mode=False, model_complexity=1)

    # ---- GStreamer pipeline (fallback to FFmpeg) ----
    gst = (
        f"rtspsrc location={RTSP_URL} latency=100 protocols=tcp ! "
        "rtph264depay ! h264parse ! v4l2h264dec ! "
        "videoconvert ! video/x-raw,format=BGR,width=640,height=480,framerate=15/1 ! "
        "appsink drop=true max-buffers=1 sync=false"
    )
    cap = cv2.VideoCapture(gst, cv2.CAP_GSTREAMER)
    if not cap.isOpened():
        logging.info("GStreamer failed → using FFmpeg")
        cap = cv2.VideoCapture(RTSP_URL, cv2.CAP_FFMPEG)
        cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

    if not cap.isOpened():
        logging.error("Cannot open RTSP stream")
        stream_active = False
        return

    ref_idx = 0
    last_time = time.time()
    frame_delay = 1.0 / 10          # advance reference 10 times/sec
    angle_thr = 20
    frame_cnt = 0

    def angle(a,b,c):
        a,b,c = np.array(a),np.array(b),np.array(c)
        ab,bc = a-b,c-b
        cos = np.dot(ab,bc)/(np.linalg.norm(ab)*np.linalg.norm(bc)+1e-6)
        return np.degrees(np.arccos(np.clip(cos,-1,1)))

    while stream_active and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            time.sleep(0.1)
            continue
        if frame_cnt % 3:               # drop 2 out of 3 frames → ~5 fps processing
            frame_cnt += 1
            continue
        frame_cnt += 1

        h,w,_ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)

        canvas = np.zeros((h,w,3), np.uint8)

        # ---- reference (blue) ----
        ref = ref_seq[ref_idx]
        for a,b in mp_pose.POSE_CONNECTIONS:
            if ref[a][3]>0.1 and ref[b][3]>0.1:
                cv2.line(canvas,
                         (int(ref[a][0]*w),int(ref[a][1]*h)),
                         (int(ref[b][0]*w),int(ref[b][1]*h)),
                         (255,0,0),2)

        # ---- user (green/red) ----
        if results.pose_landmarks:
            user = [(l.x,l.y,l.z,l.visibility) for l in results.pose_landmarks.landmark]
            for a,b in mp_pose.POSE_CONNECTIONS:
                if all(x[3]>0.1 for x in [ref[a],ref[b],user[a],user[b]]):
                    parent = next((c[0] for c in mp_pose.POSE_CONNECTIONS if c[1]==b), None)
                    if parent and ref[parent][3]>0.1 and user[parent][3]>0.1:
                        ra = angle((ref[a][0],ref[a][1]), (ref[b][0],ref[b][1]), (ref[parent][0],ref[parent][1]))
                        ua = angle((user[a][0],user[a][1]), (user[b][0],user[b][1]), (user[parent][0],user[parent][1]))
                        col = (0,255,0) if abs(ra-ua)<angle_thr else (0,0,255)
                        cv2.line(canvas,
                                 (int(user[a][0]*w),int(user[a][1]*h)),
                                 (int(user[b][0]*w),int(user[b][1]*h)),
                                 col,2)

        _, buf = cv2.imencode('.jpg', canvas, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
        if frame_queue.full():
            frame_queue.get_nowait()          # drop oldest
        frame_queue.put(buf.tobytes())

        # advance reference pose
        if time.time()-last_time >= frame_delay:
            last_time = time.time()
            ref_idx = (ref_idx+1) % len(ref_seq)

        time.sleep(0.01)

    cap.release()
    pose.close()
    stream_active = False
    logging.info("Stream worker finished")

# --------------------------------------------------  MJPEG feed  -----
@app.route('/video_feed')
def video_feed():
    def gen():
        while True:
            try:
                frame = frame_queue.get(timeout=1)
                yield (b'--frame\r\n'
                       b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            except queue.Empty:
                if not stream_active:
                    break
                continue
    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')

print("Flask routes defined – run the next cell to launch the server")

## 3. Start Flask Server (Run this cell)

In [ ]:
import threading
from werkzeug.serving import make_server

class FlaskThread(threading.Thread):
    def __init__(self):
        super().__init__(daemon=True)
        self.srv = make_server('0.0.0.0', 5000, app)
        self.ctx = app.app_context()
        self.ctx.push()

    def run(self):
        self.srv.serve_forever()

    def shutdown(self):
        self.srv.shutdown()

# launch
server_thread = FlaskThread()
server_thread.start()


In [ ]:
server_thread.shutdown()  